# Tutorial 1

ToDo: Add links on Section * below

In this tutorial you will:
- learn the Flow basics and implement your first Flow [Section 1]
- understand the basics of Flows engine and serve your first Flow [Section 2]
- learn how to call served Flows [Section 2]
- learn how to compose Flows into more complex Flows [Section 3]

In [11]:
%load_ext autoreload
%autoreload 2

# Imports

from aiflows.utils.general_helpers import read_yaml_file
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.utils.colink_utils import start_colink_server
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
import sys
sys.path.append("..")
from utils import compile_and_writefile

# Specify path of your Flow Modules --> # ToDo: Should a user know what Flow modules are and where they are located? If so split in a separate cell and epxlain in text above
FLOW_MODULES_PATH = "./"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Reverse Number Flow

In this section we will introduce the concept of a Flow and implement a simple Flow that reverses a number passed in the input message.

In a nutshell, Flows are objects that perform semantically meaningful unit of work and comunicate via messages.

There are two types of Flows: 
1) **Atomic Flows:** Performs the computation directly
2) **Composite Flows:** Orchestrates other Flows in performing the computation

When a message is send to a Flow, the Flow processes the message according to the logic specified in its `run` method. We will now go through the implementation of the `ReverseNumberAtomicFlow`.

In Section 3, we see provide an example implementation of a Composite Flow.

In [14]:
%%compile_and_writefile ReverseNumberAtomic.py


from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage

class ReverseNumberAtomicFlow(AtomicFlow):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    # The run method defines the main logic of the Flow that is executed for every input message
    def run(self, input_message: FlowMessage):
        ## ~~~~ Getting the input ~~~~

        # Get the input dictionary from the input message
        input_data = input_message.data
        
        # Get the input number from the data dictionary (int)
        input_number = input_data["number"]
        

        ## ~~~~ Main logic ~~~~
        ########## YOUR CODE GOES HERE ##########
        # TODO: Reverse the input number (e.g. 1234 -> 4321)
        reversed_number = int(str(input_number)[::-1])
        ########## YOUR CODE GOES HERE ##########
        
        # ~~~ Preparing the response message ~~~
        response = {"reversed_number": reversed_number}
        
        # This method packages the `response` in a FlowMessage object 
        # containing the necessary metadata to send the message back
        # to the sender of the input message 
        reply = self.package_output_message(
            input_message=input_message,
            response=response,
        )
        
        # ~~~ Sending the response ~~~
        self.send_message(
            reply,
            is_reply=True,
        )


ToDo: Explain why is the below necessary and what does each field do

In [4]:
default_config_reverse_number = \
{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",

    # TODO: Define the target
    "_target_": "ReverseNumberAtomic.ReverseNumberAtomicFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "reversed_number",
}

## 2. Flows Engine

In this section we introduce the basics of the Flows Engine, and guide you through the process of serving and accessing served Flows.

The Flows Engine is build on top of CoLink (ToDo: Add Link to them). ToDo: Explain why do we need this and what does this enable and the basic concepts in as few words as possible


## 2.1 Connect to the CoLink Server (ToDo: Connect or start? Or connect and explain that we are starting since ...)

ToDo: Few words of explanation: what's the point, why does this matter?

### 2.1.1 Connection to an existing server

ToDo: Explain setting

In [56]:
# ToDo

### 2.1.1 Staring a CoLink server and conecting to it

ToDo: Explain setting

In [57]:
cl = colink_utils.start_colink_server()

## 2.2 Serving the Reverse Number Flow

ToDo: Few words of explanation: what's the point, why does this matter? Explain any fields that are critical to understand (critical as in even a newbie should know).

In [9]:
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberAtomicFlow_served", #reference name we give to flow
    default_config=default_config_reverse_number,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    serve_mode="statefull",
)

ReverseNumberAtomicFlow_served is already being served at flows:ReverseNumberAtomicFlow_served


False

## 2.3 Getting an instance of the Reverse Number Flow

ToDo: Few words of explanation: what's the point, why does this matter? Explain any fields that are critical to understand (critical as in even a newbie should know).

In [6]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

# Get an instance of the flow
proxy_reverse_number_flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberAtomicFlow_served", #???
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)

Dispatch worker started in attached thread.
Mounted 35acecad-eb36-4b5d-a383-25813468d876 at flows:ReverseNumberAtomicFlow_served:mounts:local:35acecad-eb36-4b5d-a383-25813468d876


### 2.4. Call the Reverse Number Flow via its proxy Flow

ToDo: Few words of explanation: what's the point, why does this matter?

In [8]:
input_data = {"id": 0, "number": 12345}

# ~~~ Prepare the input message ~~~
input_message: FlowMessage = proxy_reverse_number_flow.package_input_message(input_data)

# ~~~ Send message to the Flow and ask for a future ~~~
future = proxy_reverse_number_flow.get_reply_future(input_message)

# ~~~ Get the response from the future ~~~
reply_data = future.get_data()  # This is a blocking call (with aiFlows there are multiple ways to send messages as we shall see in the following tutorials)
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")



KeyboardInterrupt: 

## 3. Reverse Number Sequential

In this section we will implement an example of a Composite Flow, `ReverseNumberSequential` that calls the `ReverseAtomicFlow` twice -- reversing the reversed number -- resulting in the same number
We serve it and use it in an example.

# ToDo:
- motivating example (why are you telling me this)
- different message sending primitives (and why should I care)
- syncronous implementation then async implementation (highlight benefits)
- explanations in between as above

### 3.1 Writing the Reverse Number Sequential Flow Class (ACTION REQUIRED) 

In [ ]:
%%compile_and_writefile ReverseNumberSequential.py


from aiflows.base_flows import CompositeFlow
from aiflows.messages import FlowMessage
from aiflows.interfaces import KeyInterface
class ReverseNumberSequentialFlow(CompositeFlow):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #~~~~~~~~~~~ Key Transformation solution 1 ~~~~~~~~~~~
        self.input_interface_second_reverse_flow = KeyInterface(
            keys_to_rename= {"reversed_number": "number"},
            keys_to_select= ["number"],
        )
    
        self.ouput_interface_reply = KeyInterface(
            keys_to_rename= {"reversed_number": "output_number"},
            keys_to_select = ["output_number"],
        )
        
        self.get_next_call = {
            "first_reverse_flow": "second_reverse_flow",
            "second_reverse_flow": "reply_to_message",
            "reply_to_message": "first_reverse_flow"
        }
    def set_up_flow_state(self):
        super().set_up_flow_state()
        self.flow_state = {"flow_to_call": "first_reverse_flow"}
        
    def get_next_flow_to_call(self):
        return self.get_next_call[self.flow_state["flow_to_call"]]
        
    def run(self, input_message: FlowMessage):
        # The run method in this case needs to make decisions depending on the state of the Flow (the state of the computation)
        # You can think of this as an state machine.
        # For this Flow we implement this like a switch statment. 

        # We then call the next flow and pass the input message to it. which we expect to get a reply from 
        # back in the input queue (which will call the run method againg)
        flow_to_call = self.flow_state["flow_to_call"]
        
        #Case where we need to reverse the number for the first time
        if flow_to_call == "first_reverse_flow":
            #Save the initial message to the state
            self.flow_state["initial_message"] = input_message
            
            #Calls the first flow and requests a reply to be sent back to the input queue 
            # (The queue to send back to is specified by self.get_instance_id() --> id of this flow instance
            # of ReverseNumberSequentialFlow)
            self.subflows["first_reverse_flow"].get_reply(
                input_message,
                self.get_instance_id()
            )
        
        #Case where we need to reverse the number for the second time
        elif flow_to_call == "second_reverse_flow":
            
            #Applies a transformation to the input message (renames keys of dictonary so that they match the
            # required format of the second flow)
            message = self.input_interface_second_reverse_flow(input_message)
            
            #TODO: Call the second flow and requests a reply to be sent back to the input queue
            self.subflows["second_reverse_flow"].
        
        #Case where we need to reply to the initial message (we've already reversed the number twice)
        else:
            message = self.ouput_interface_reply(input_message)
            
            #package ouput message to send back
                #This method packages `response` in a FlowMessage object 
                # containing the necessary metadata to send the message back
                # to the sender of the input message. 
            reply = self.package_output_message(
                input_message = self.flow_state["initial_message"],
                response = message.data
            )
            #send back the reply to initial caller of the flow
            self.send_message(reply, is_reply = True)
            
        self.flow_state["flow_to_call"] = self.get_next_flow_to_call()

In [ ]:
default_config_reverse_number_sequential = \
{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",

    # TODO: Define the target
    "_target_": "ReverseNumberSequential.ReverseNumberSequentialFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "output_number",
    
    "subflows_config": {
        "first_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "A proxy flow that calls reverse number to reverse number AGAIN.",
            "description": "A proxy flow that calls reverse number to reverse number.",
        },
        "second_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "Proxy Second Reverse",
            "description": "A proxy flow that calls reverse number to reverse number AGAIN.",
        },
    }
}


### 3.2 Serving & Getting and Instance of the Reverse Number Flow

#### 3.2.1 Serving the Reverse Number Sequential Flow

In [ ]:
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberSequentialFlow", #reference name we give to flow
    default_config=default_config_reverse_number_sequential,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    serve_mode="statefull",
)

#### 3.2.2 Getting an instance of the Reverse Number Flow

In [ ]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

# Get an instance of the flow
proxy_reverse_number_sequential_flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberSequentialFlow", #???
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)

### 3.3. Call the Reverse Number Sequential Flow via the Proxy

In [ ]:
input_data = {"id": 0, "number": 12345}

# Package your data in a Flow Message

## Option 1: Via the FlowMessage class
# input_message = FlowMessage(
#     data=input_data,
# )

## Option 2 (prefered): Via the package input message method
input_message = proxy_reverse_number_sequential_flow.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_number_sequential_flow.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()
#To get the response as a FlowMessage object
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")